# Yosys inference of memories

This example demonstrates direct inference of a dual port (simplex) memory description into a FPGA hard memory block (here: Lattice ECP5 DP16KD primitive).
It requires a recent (>= v0.12) release of yosys using the improved memory mapping.

In [1]:
import sys
sys.path.insert(0, '../../')

In [2]:
from myirl.emulation.myhdl import *
from myirl.test.test_array import r1w1, SigArray

## Simple read and write port memory

This implementation uses bypass logic when a write is occuring during a read from the same address.

In [3]:
@block
def r1w1(
        clk : ClkSignal,
        we  : Signal,
        addr_r: Signal,
        addr_w: Signal,
        din: Signal,
        dout: Signal.Output,
        ram : SigArray,
        MODE = False, # Conditional compilation flag w/o type annotation
        DWIDTH=16, AWIDTH=10,
        TRANSPARENCY = False
    ) -> IRL:

    print("CALLED MEMORY INSTANCE", DWIDTH)
    
    if TRANSPARENCY:
        @always(clk.posedge)
        def mem_rw_transparent():
            if we and addr_w == addr_r:
                dout.next = din  #Forward
            else:
                dout.next = ram[addr_r][DWIDTH:]

            if we:
                ram[addr_w].next = din
    else:
        @always(clk.posedge)
        def mem_rw():
            dout.next = ram[addr_r][DWIDTH:]
            
            if we:
                ram[addr_w].next = din        
        
    return instances()


In [4]:
def memtest(MODE = 0, STYLE = 1, data_w=16, addr_w=6, mem = r1w1, TRANSPARENT = True):
    c = ClkSignal(name = 'clk')
    c.init = True
    wren = Signal(bool(), name = 'we')
    ra, wa = [ Signal(intbv()[addr_w:]) for n in ['addr_write', 'addr_read'] ]
    a, q = [ Signal(intbv()[data_w:]) for n in ['a', 'q'] ]

    ram_data = SigArray([ intbv(v)[data_w:] for v in range(2 ** addr_w)],
        name='ram_sig', init=True)

    inst = mem(clk=c, we=wren, addr_r=ra, addr_w=wa, din=a, dout=q, MODE=False, ram = ram_data,
        AWIDTH=addr_w, DWIDTH=data_w, TRANSPARENCY = TRANSPARENT)

    return inst

## Mapping to hardware

In [5]:
from myirl.targets import pyosys

In [6]:
DATA_WIDTH = 64

Elaborate memory unit and emit CXXRTL code:

In [7]:
def convert(dw, aw = 7):

    tgt = pyosys.RTLIL("memtest%d" % dw)

    MEM = r1w1

    tb = memtest(data_w=dw, mem = MEM, addr_w = aw, TRANSPARENT = True)
    d = tb.elab(tgt, elab_all = True)
    d = d[0]
    d.run("hierarchy -check")
    d.run("stat", capture = None)
    # d.run("write_rtlil mem8.il")
    d.run("debug memory -nomap; debug opt")
    
    return d

In [8]:
d = convert(DATA_WIDTH, 7)

CALLED MEMORY INSTANCE 64
Creating process 'r1w1/mem_rw_transparent' with sensitivity (clk'rising,)
 Adding module with name `r1w1` 
 DEBUG: SKIP ARRAYTYPE `ram` : <class 'myirl.lists.SigArray'> 
     DEBUG SLICE READ MEMORY to dout AWIDTH:7 DWIDTH:64
 [Component 'meminit/meminit'] blackbox not returning instances 
DEBUG ADD ROM 7:64
 PARAM PRIORITY --> 48 
 PARAM WORDS --> 128 
 PARAM ABITS --> 7 
 PARAM WIDTH --> 64 
 PARAM MEMID --> mem_ram_sig 
 [Component 'memrd/memrd'] blackbox not returning instances 
 PARAM CLK_ENABLE --> False 
 PARAM CLK_POLARITY --> True 
 PARAM TRANSPARENT --> False 
 PARAM ABITS --> 7 
 PARAM WIDTH --> 64 
 PARAM MEMID --> mem_ram_sig 
     DEBUG SLICE WRITE MEMORY to ram_sig AWIDTH:7 DWIDTH:64
 [Component 'memwr/memwr'] blackbox not returning instances 
 PARAM CLK_ENABLE --> True 
 PARAM CLK_POLARITY --> True 
 PARAM PRIORITY --> 48 
 PARAM ABITS --> 7 
 PARAM WIDTH --> 64 
 PARAM MEMID --> mem_ram_sig 
DEBUG TOP LEVEL WRITE MEM ram_sig
 FINALIZE implemen

## Testing hardware generation

Note: Post map simulation may require external memory models of blackbox Vendor Primitives.

In [9]:
TECHMAP = '/usr/share/yosys'

def synth_ecp5(d):
    # Read blackbox cells for awareness of DP16KD:
    d.run("read_verilog -lib -specify %s/ecp5/cells_sim.v %s/ecp5/cells_bb.v" % (TECHMAP, TECHMAP))

    # First try DP16KD mapping:
    d.run("memory_bram -rules %s/ecp5/brams.txt" % TECHMAP)
    d.run("techmap -map %s/ecp5/brams_map.v" % TECHMAP)

    # Remaining (addr_w <= 8 bit) to LUT RAM:
    d.run("memory_bram -rules %s/ecp5/lutrams.txt" % TECHMAP)
    d.run("techmap -map %s/ecp5/lutrams_map.v" % TECHMAP)
    d.run("opt_clean")
    
def synth_gatemate(d):
    # Read blackbox cells for awareness of DP16KD:
    d.run("read_verilog -lib -specify %s/gatemate/cells_sim.v %s/gatemate/cells_bb.v" % (TECHMAP, TECHMAP))

    d.run("memory_bram -rules %s/gatemate/brams.txt" % TECHMAP)
    d.run("techmap -map %s/gatemate/brams_map.v" % TECHMAP)

    d.run("opt")
    
def synth_efinix(d):
    # Read blackbox cells for awareness of DP16KD:
    d.run("read_verilog -lib -specify %s/efinix/cells_sim.v %s/efinix/cells_map.v" % (TECHMAP, TECHMAP))

    d.run("memory_bram -rules %s/efinix/brams.txt" % TECHMAP)
    d.run("techmap -map %s/efinix/brams_map.v" % TECHMAP)

    d.run("opt")    
    
d.run("ls", capture = None)

synth_ecp5(d)
d.run("stat", capture = None)

d.run("write_rtlil mapped.il")

d.display_rtl(selection = '*', fmt = 'dot')


-- Running command `ls' --

1 modules:
  r1w1

-- Running command `tee -q read_verilog -lib -specify /usr/share/yosys/ecp5/cells_sim.v /usr/share/yosys/ecp5/cells_bb.v' --

-- Running command `tee -q memory_bram -rules /usr/share/yosys/ecp5/brams.txt' --

-- Running command `tee -q techmap -map /usr/share/yosys/ecp5/brams_map.v' --

-- Running command `tee -q memory_bram -rules /usr/share/yosys/ecp5/lutrams.txt' --

-- Running command `tee -q techmap -map /usr/share/yosys/ecp5/lutrams_map.v' --

-- Running command `tee -q opt_clean' --

-- Running command `stat' --

13. Printing statistics.

=== r1w1 ===

   Number of wires:                 12
   Number of wire bits:            347
   Number of public wires:           6
   Number of public wire bits:     144
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                  7
     $and                            1
     $dff                          

In [10]:
from yosys import display
display.display_dot("memtest%d" % DATA_WIDTH)

In [11]:
class GateMateTarget(pyosys.RTLIL):
    def __init__(self, *args):
        super().__init__(*args)
        print("Selecting GateMate target")
        self.debug = True
        
    def finalize(self, top):
        print("GATEMATE FINALIZE")
        tname = top.name
        design = self._design
        design.run("hierarchy -top %s" % tname)
        synth_gatemate(design)
        design.run('stat', capture = None)
        # self.write_cxxrtl(top)


In [12]:
import simulation
from yosys.simulator import CXXRTL


@simulation.sim.testbench(CXXRTL, time_unit = 'ns', target_class = GateMateTarget)
def tb_memtest(MODE = 0, STYLE = 1, data_w=16, addr_w=7, mem = r1w1):
    
    ClkSignal = simulation.ClkSignal
    Signal = simulation.Signal

    c = ClkSignal(name = 'clk')
    c.init = True
    
    M = (1 << (data_w - 1))

    wren = Signal(bool(), name = 'we')
    ra, wa = [ Signal(intbv()[addr_w:]) for n in ['addr_write', 'addr_read'] ]
    a, q = [ Signal(intbv()[data_w:]) for n in ['a', 'q'] ]

    ram_sig = SigArray([intbv(M | v)[data_w:] for v in range(2 ** addr_w)],
        name='ram_sig', init=True)

    # print(ram_sig[0].size())

    inst = mem(clk=c,
        we=wren,
        addr_r=ra, addr_w=wa, din=a, dout=q, MODE=False, ram = ram_sig,
        AWIDTH=addr_w, DWIDTH=data_w,
        TRANSPARENCY = True)

    @simulation.always(simulation.delay(2))
    def clkgen():
        c.next = ~c

    def write(addr, data):
        print("WRITE", addr, data)
        yield c.negedge
        wa.next = addr
        a.next = data
        wren.next = True
        yield c.negedge
        wren.next = False

    def write_verify(addr, data, value):
        print("WRITE VERIFY", addr)
        yield c.negedge
        wa.next = addr
        a.next = data
        wren.next = True
        yield c.negedge
        # print("DEBUG Q", bin(int(q)))
        assert int(q) == value
        wren.next = False

    @simulation.sequence
    def stim():
        ra.next = 0x20
        wa.next = 0x00

        # Write and verify that we're not bypassing:
        # i.e. the value is expected to be the initial one
        yield from write_verify(0x00, 0xaa, M | 0x20)
        ra.next = 0x00
        yield c.negedge
        # Now make sure we're getting the written value
        assert int(q) == 0xaa

        # ra == wa, verify transparency bypass:
        ra.next = 0x40
        yield from write_verify(0x40, 0x55, 0x55)
        yield c.negedge
        ra.next = 0x00
        yield c.negedge
        assert int(q) == 0xaa
        ra.next = 0x40
        yield c.negedge
        assert int(q) == 0x55
        ra.next = 0x1a
        yield c.negedge
        assert int(q) == M | 0x1a # Initial

        print("SIM DONE")

    return simulation.instances()


In [13]:
r1w1.ctx.components

['r1w1_s1_s1_s7_s7_s64_s64_s128_0_64_7_1']

In [14]:
t = tb_memtest(data_w = 12)

 Module top_r1w1: Existing instance r1w1, rename to r1w1_1 
CALLED MEMORY INSTANCE 12
Creating process 'r1w1/mem_rw_transparent' with sensitivity (clk'rising,)
DEBUG: Skip non-simulation type <class 'myirl.emulation.myhdl.wrapped_wrapper'>
DEBUG: Skip non-simulation type <class 'type'>
DEBUG: Skip non-simulation type <class 'myirl.lists.SigArray'>
DEBUG: Skip non-simulation type <class 'function'>
DEBUG: Skip non-simulation type <class 'type'>
DEBUG: Skip non-simulation type <class 'function'>


In [15]:
t.run(100)
t.design.run("write_rtlil memtest.il")

Selecting GateMate target
DEBUG DUMMY GET s_72ae
 Adding module with name `r1w1_1` 
 DEBUG: SKIP ARRAYTYPE `ram` : <class 'myirl.lists.SigArray'> 
     DEBUG SLICE READ MEMORY to dout AWIDTH:7 DWIDTH:12
DEBUG ADD ROM 7:12
 PARAM PRIORITY --> 48 
 PARAM WORDS --> 128 
 PARAM ABITS --> 7 
 PARAM WIDTH --> 12 
 PARAM MEMID --> mem_ram_sig 
 PARAM CLK_ENABLE --> False 
 PARAM CLK_POLARITY --> True 
 PARAM TRANSPARENT --> False 
 PARAM ABITS --> 7 
 PARAM WIDTH --> 12 
 PARAM MEMID --> mem_ram_sig 
     DEBUG SLICE WRITE MEMORY to ram_sig AWIDTH:7 DWIDTH:12
 PARAM CLK_ENABLE --> True 
 PARAM CLK_POLARITY --> True 
 PARAM PRIORITY --> 48 
 PARAM ABITS --> 7 
 PARAM WIDTH --> 12 
 PARAM MEMID --> mem_ram_sig 
DEBUG TOP LEVEL WRITE MEM ram_sig
GATEMATE FINALIZE

-- Running command `tee -q hierarchy -top r1w1_1' --

-- Running command `tee -q read_verilog -lib -specify /usr/share/yosys/gatemate/cells_sim.v /usr/share/yosys/gatemate/cells_bb.v' --

-- Running command `tee -q memory_bram -rules /

# Duplex read, single write

**Attention**: The pyosys API may crash the notebook without warning (terminate the kernel) when the yosys-abc utility is not present. May be the case in the Binder setup.

In [16]:
# ! [ `which yosys-abc` ] || sudo apt-get install yosys-ghdl

In [17]:
@block
def r2w1_gated(
        clk : ClkSignal,
        re	: Signal,
        we	: Signal,
        addr_r0: Signal,
        addr_r1: Signal,
        addr_w: Signal,
        din: Signal,
        dout0: Signal.Output,
        dout1: Signal.Output,
        ram_data : list,
        MODE = False, # Conditional compilation flag w/o type annotation
        DWIDTH=16, AWIDTH=10
    ) -> IRL:

    ram = SigArray(ram_data, name='ram_sig', init=True)

    @always(clk.posedge)
    def mem_r2w():
        dout1.next = ram[addr_r1]
        if re:
            dout0.next = ram[addr_r0]
        if we:
            ram[addr_w].next = din

    return instances()


In [18]:
@simulation.sim.testbench(CXXRTL, time_unit = 'ns', target_class = GateMateTarget)
def tb_memtest2(MODE = 0, STYLE = 1, data_w=16, addr_w=6, mem = r2w1_gated):
    c = simulation.ClkSignal(name = 'clk')
    c.init = True
    rden = simulation.Signal(bool(), name = 're')
    wren = simulation.Signal(bool(), name = 'we')
    ra, rb, wa = [ simulation.Signal(intbv()[addr_w:]) for _ in range(3) ]
    a, q0, q1 = [ simulation.Signal(intbv()[data_w:]) for n in ['a', 'q0', 'q1'] ]

    ram_data = [ intbv(v)[data_w:] for v in range(2 ** addr_w)]

    inst = mem(clk=c,
        re=rden,
        we=wren,
        addr_r0=ra,
        addr_r1=rb,
        addr_w=wa, din=a,
        dout0=q0, dout1=q1,
        MODE=False, ram_data = ram_data,
        AWIDTH=addr_w, DWIDTH=data_w)

    @simulation.always(simulation.delay(2))
    def clkgen():
        c.next = ~c

    @simulation.sequence
    def stim():
        rb.next = 0xfa
        ra.next = 0xfa
        wa.next = 0x20
        wren.next = False
        yield simulation.delay(5)
        yield c.negedge
        assert int(q1) == 0xfa
        assert int(q0) != 0xfa
        rden.next = True
        yield c.negedge
        ra.next = 0xaa
        rden.next = False

        wren.next = True
        a.next = 0x44
        yield c.negedge
        wren.next = False
        yield c.negedge
        assert int(q0) == 0xfa
        yield c.negedge
        rb.next = 0x20
        print(int(q0), int(q1))
        yield c.negedge
        print(int(q0), int(q1))
        assert int(q1) == 0x44

        print("SIM DONE")

    return simulation.instances()


In [19]:
dw = 16

MEM = r2w1_gated

tb = tb_memtest2(data_w=dw, mem = MEM, addr_w = 11)
tb._force_compile = True
tb.debug()
# tb.run(200)


Creating process 'r2w1_gated/mem_r2w' with sensitivity (clk'rising,)
DEBUG: Skip non-simulation type <class 'myirl.emulation.myhdl.wrapped_wrapper'>
DEBUG: Skip non-simulation type <class 'list'>
Selecting GateMate target
DEBUG DUMMY GET s_bb42
 Adding module with name `r2w1_gated` 
 DEBUG: SKIP NON-SIGNAL ARGUMENT `ram_data` : <class 'list'> 
 DEBUG SLICE READ MEMORY to dout1 AWIDTH:11 DWIDTH:16
     DEBUG SLICE READ MEMORY to dout0 AWIDTH:11 DWIDTH:16
     DEBUG SLICE WRITE MEMORY to ram_sig AWIDTH:11 DWIDTH:16
DEBUG ADD ROM 11:16
 PARAM PRIORITY --> 48 
 PARAM WORDS --> 2048 
 PARAM ABITS --> 11 
 PARAM WIDTH --> 16 
 PARAM MEMID --> mem_ram_sig 
 PARAM CLK_ENABLE --> True 
 PARAM CLK_POLARITY --> True 
 PARAM TRANSPARENT --> False 
 PARAM ABITS --> 11 
 PARAM WIDTH --> 16 
 PARAM MEMID --> mem_ram_sig 
DEBUG TOP LEVEL READ MEM dout1
 PARAM CLK_ENABLE --> True 
 PARAM CLK_POLARITY --> True 
 PARAM TRANSPARENT --> False 
 PARAM ABITS --> 11 
 PARAM WIDTH --> 16 
 PARAM MEMID --> mem_

In [20]:
tb.design.run("write_rtlil r2w1.il", capture = None)


-- Running command `write_rtlil r2w1.il' --

34. Executing RTLIL backend.
